## Import modules

In [2]:

import shutil, contextlib, os, time, re, subprocess
from huggingface_hub import snapshot_download
from pathlib import Path
from chatterbox.models.voice_encoder import VoiceEncoder
import torchaudio as ta
from IPython.display import Audio
import torch
from safetensors.torch import load_file, save_file
from chatterbox.mtl_tts import ChatterboxMultilingualTTS
from chatterbox.mtl_tts import Conditionals


# Set device

In [3]:
has_cuda = torch.cuda.is_available()
has_mps = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()

if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    fp16 = False
    no_cuda = True
elif torch.cuda.is_available():
    device = "cuda"
    fp16 = True
    no_cuda = False
else:
    device = "cpu"
    fp16 = False
    no_cuda = True

print("Device:", device)

Device: cuda


## Download Model into Local Directory

In [4]:
# ---- Choose the exact base you fine-tuned from
REPO_ID = "ResembleAI/chatterbox"

# ---- Project dirs (…/models/{base,checkpoints,hf_cache})
parent_dir = Path(os.getcwd()).parent
models_folder = parent_dir / "models"
base_folder = models_folder / "base" / REPO_ID.split("/")[-1]
checkpoints_folder = models_folder / "checkpoints"
cache_folder  = models_folder / "hf_cache"

for p in (base_folder, checkpoints_folder, cache_folder):
    p.mkdir(parents=True, exist_ok=True)

# ---- Allow filename variants seen across releases
allow = [
    "ve.pt", "ve.safetensors",
    "s3gen.pt",
    "conds.pt",
    "Cangjie5_TC.json",
    # tokenizer variants
    "mtl_tokenizer.json", "tokenizer.json", "grapheme_mtl_merged_expanded_v1.json",
    # T3 variants
    "t3_23lang.safetensors", "t3_mtl23ls_v2.safetensors",
]

ckpt_dir = Path(snapshot_download(
    repo_id=REPO_ID,
    repo_type="model",
    revision="main",
    allow_patterns=allow,
    token=os.getenv("HF_TOKEN"),
    cache_dir=str(cache_folder),
    local_dir=str(base_folder),           # <- save into models/base/<repo>
    local_dir_use_symlinks=False          # <- write real files
))

print("Base model saved to:", ckpt_dir)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 576.05it/s]

Base model saved to: /nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/models/base/chatterbox


## Preprocess data

In [5]:
output_dir = "../models/checkpoints/chatterbox_finetuned_swedish"
local_model_dir = str(ckpt_dir)
metadata_file = "../data/david/metadata.txt"
dataset_dir = "../data/david"
dataloader_num_workers = 0
precomputed_output = "../data/david/precomputed_t3_features.pt"

Path(output_dir).mkdir(parents=True, exist_ok=True)

print("Using python at:")
subprocess.run(["which", "python3"], check=True)

# environment (reuse what you already have if you want)
env = os.environ.copy()

# 1) RUN PREPROCESSING SCRIPT ONCE
preprocess_cmd = [
    "python3", "../scripts/preprocess_data.py",
    "--metadata_file", metadata_file,
    "--local_model_dir", local_model_dir,
    "--output_path", precomputed_output,
    "--language_id", "sv",
]

print("Running preprocessing:", " ".join(preprocess_cmd))
subprocess.run(preprocess_cmd, env=env, check=True)

print("✅ Preprocessing done, saved to", precomputed_output)

Using python at:
/nobackup/joeri765/venvs/pni-py310/bin/python3
Running preprocessing: python3 ../scripts/preprocess_data.py --metadata_file ../data/david/metadata.txt --local_model_dir /nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/models/base/chatterbox --output_path ../data/david/precomputed_t3_features.pt --language_id sv


/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/perth/perth_net/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Metadata file: ../data/david/metadata.txt
Local model dir: /nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/models/base/chatterbox
Output path: ../data/david/precomputed_t3_features.pt
Language ID: sv


/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


loaded PerthNet (Implicit) at step 250,000
Audio file does not exist: /nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/data/david/soundfiles_david_sentence05.wav
Loaded 88 lines from metadata


Precomputing T3 features: 100%|██████████| 88/88 [00:27<00:00,  3.22it/s]


Saved 88 examples to ../data/david/precomputed_t3_features.pt
✅ Preprocessing done, saved to ../data/david/precomputed_t3_features.pt


## Finetune

In [ ]:
import os
import subprocess
from pathlib import Path

ckpt_dir = Path("../models/base/chatterbox")  # same dir you used for preprocessing
output_dir = "../models/checkpoints/chatterbox_finetuned_swedish"
precomputed_file = "../data/david/precomputed_t3_features.pt"
dataloader_num_workers = 0

# Device helpers (if you already have has_mps / no_cuda defined, reuse them)
has_mps = torch.backends.mps.is_available() if "torch" in globals() else False
no_cuda = False  # set True if you want to force CPU

Path(output_dir).mkdir(parents=True, exist_ok=True)

print("Using python at:")
subprocess.run(["which", "python3"], check=True)

cmd = [
    "python3", "../scripts/finetune_mtl_precomputed.py",
    "--output_dir", output_dir,
    "--local_model_dir", str(ckpt_dir),
    "--precomputed_path", precomputed_file,
    "--eval_split_size", "0.1",
    "--num_train_epochs", "20",
    "--per_device_train_batch_size", "2",
    "--gradient_accumulation_steps", "4",
    "--learning_rate", "1e-5",
    "--warmup_steps", "200",
    "--weight_decay", "0.01",
    "--max_grad_norm", "1.0",
    "--logging_steps", "10",
    "--evaluation_strategy", "steps",
    "--eval_steps", "500",
    "--save_strategy", "steps",
    "--save_steps", "500",
    "--save_total_limit", "5",
    "--report_to", "tensorboard",
    "--do_train",
    "--do_eval",
    "--dataloader_pin_memory", "False",
    "--eval_on_start", "True",
    "--label_names", "labels_speech",
    "--language_id", "sv",
    "--dataloader_num_workers", str(dataloader_num_workers),
    "--seed", "42",
    "--fp16",
    "--lr_scheduler_type", "cosine",
    "--early_stopping_patience", "3",
    "--load_best_model_at_end", "True",
    "--metric_for_best_model", "eval_loss",
    "--greater_is_better", "False",
]

env = os.environ.copy()
if has_mps:
    env["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
if no_cuda:
    cmd.append("--no_cuda")

print("Running:", " ".join(cmd))
subprocess.run(cmd, env=env, check=True)

Using python at:
Running: python3 ../scripts/finetune_mtl_precomputed.py --output_dir ../models/checkpoints/chatterbox_finetuned_swedish --local_model_dir ../models/base/chatterbox --precomputed_path ../data/david/precomputed_t3_features.pt --eval_split_size 0.1 --num_train_epochs 20 --per_device_train_batch_size 2 --gradient_accumulation_steps 4 --learning_rate 1e-5 --warmup_steps 200 --weight_decay 0.01 --max_grad_norm 1.0 --logging_steps 10 --evaluation_strategy steps --eval_steps 500 --save_strategy steps --save_steps 500 --save_total_limit 5 --report_to tensorboard --do_train --do_eval --dataloader_pin_memory False --eval_on_start True --label_names labels_speech --language_id sv --dataloader_num_workers 0 --seed 42 --fp16 --gradient_checkpointing True --lr_scheduler_type cosine --early_stopping_patience 3 --load_best_model_at_end True --metric_for_best_model eval_loss --greater_is_better False
/nobackup/joeri765/venvs/pni-py310/bin/python3


/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/perth/perth_net/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
11/24/2025 10:58:09 - INFO - __main__ - Training/evaluation parameters CustomTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0

loaded PerthNet (Implicit) at step 250,000


11/24/2025 10:58:14 - INFO - __main__ - Voice Encoder frozen.
11/24/2025 10:58:14 - INFO - __main__ - S3Gen model frozen.
11/24/2025 10:58:14 - INFO - __main__ - T3 model set to trainable.
11/24/2025 10:58:14 - INFO - __main__ - Loading precomputed features from: ../data/david/precomputed_t3_features.pt
11/24/2025 10:58:14 - INFO - __main__ - Total precomputed examples: 88
11/24/2025 10:58:14 - INFO - __main__ - Train examples: 80 | Eval examples: 8
11/24/2025 10:58:15 - INFO - __main__ - *** Training Multilingual T3 model on precomputed data ***
Traceback (most recent call last):
  File "/nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/notebooks/../scripts/finetune_mtl_precomputed.py", line 268, in <module>
    main()
  File "/nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/notebooks/../scripts/finetune_mtl_precomputed.py", line 203, in main
    train_result = trainer.train(
  File "/nobackup/joeri765/venvs/pni-py310/lib/python3.10/sit

CalledProcessError: Command '['python3', '../scripts/finetune_mtl_precomputed.py', '--output_dir', '../models/checkpoints/chatterbox_finetuned_swedish', '--local_model_dir', '../models/base/chatterbox', '--precomputed_path', '../data/david/precomputed_t3_features.pt', '--eval_split_size', '0.1', '--num_train_epochs', '20', '--per_device_train_batch_size', '2', '--gradient_accumulation_steps', '4', '--learning_rate', '1e-5', '--warmup_steps', '200', '--weight_decay', '0.01', '--max_grad_norm', '1.0', '--logging_steps', '10', '--evaluation_strategy', 'steps', '--eval_steps', '500', '--save_strategy', 'steps', '--save_steps', '500', '--save_total_limit', '5', '--report_to', 'tensorboard', '--do_train', '--do_eval', '--dataloader_pin_memory', 'False', '--eval_on_start', 'True', '--label_names', 'labels_speech', '--language_id', 'sv', '--dataloader_num_workers', '0', '--seed', '42', '--fp16', '--gradient_checkpointing', 'True', '--lr_scheduler_type', 'cosine', '--early_stopping_patience', '

In [4]:
output_dir = "../models/checkpoints/chatterbox_finetuned_swedish"
local_model_dir = str(ckpt_dir)
metadata_file = "../data/david/metadata.txt"
dataset_dir = "../data/david"
dataloader_num_workers = 0

Path(output_dir).mkdir(parents=True, exist_ok=True)

print("Using python at:")
subprocess.run(["which", "python3"], check=True)

cmd = [
    "python3", "../scripts/finetune_mtl.py",
    "--output_dir", output_dir,
    "--local_model_dir", local_model_dir,
    "--metadata_file", metadata_file,
    "--dataset_dir", dataset_dir,
    "--train_split_name", "train",
    "--eval_split_size", "0.1",
    "--num_train_epochs", "10",
    "--per_device_train_batch_size", "2",
    "--gradient_accumulation_steps", "4",
    "--learning_rate", "1e-5",
    "--warmup_steps", "200",
    "--weight_decay", "0.01",
    "--max_grad_norm", "1.0",
    "--logging_steps", "10",
    "--evaluation_strategy", "epoch",
    "--save_strategy", "epoch",
    "--save_total_limit", "5",
    "--report_to", "tensorboard",
    "--do_train",
    "--do_eval",
    "--dataloader_pin_memory", "False",
    "--eval_on_start", "True",
    "--label_names", "labels_speech",
    "--text_column_name", "text_scribe",
    "--language_id", "sv",
    "--dataloader_num_workers", str(dataloader_num_workers),
    "--seed", "42",
]

# add flags from device logic
env = os.environ.copy()
if has_mps:
    env["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
if no_cuda:
    cmd.append("--no_cuda")

print("Running:", " ".join(cmd))
subprocess.run(cmd, env=env, check=True)

Using python at:
/nobackup/joeri765/venvs/pni-py310/bin/python3
Running: python3 ../scripts/finetune_mtl.py --output_dir ../models/checkpoints/chatterbox_finetuned_swedish --local_model_dir /nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/models/base/chatterbox --metadata_file ../data/david/metadata.txt --dataset_dir ../data/david --train_split_name train --eval_split_size 0.1 --num_train_epochs 10 --per_device_train_batch_size 2 --gradient_accumulation_steps 4 --learning_rate 1e-5 --warmup_steps 200 --weight_decay 0.01 --max_grad_norm 1.0 --logging_steps 10 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 5 --report_to tensorboard --do_train --do_eval --dataloader_pin_memory False --eval_on_start True --label_names labels_speech --text_column_name text_scribe --language_id sv --dataloader_num_workers 0 --seed 42


/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/perth/perth_net/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
11/16/2025 11:00:25 - INFO - __main__ - Training/evaluation parameters CustomTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0

loaded PerthNet (Implicit) at step 250,000


11/16/2025 11:01:03 - INFO - __main__ - Voice Encoder frozen.
11/16/2025 11:01:03 - INFO - __main__ - S3Gen model frozen.
11/16/2025 11:01:03 - INFO - __main__ - T3 model set to trainable.
11/16/2025 11:01:03 - INFO - __main__ - Loading and processing dataset...
11/16/2025 11:01:03 - WARNING - __main__ - Audio file not found: ../data/david/soundfiles_david_sentence05.wav (line 5). Skipping.
11/16/2025 11:01:04 - INFO - __main__ - *** Training Multilingual T3 model ***
  0%|          | 0/100 [00:00<?, ?it/s]/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)

  0%|          | 0/2 [00:00<?, ?it/s]
                                       6it/s]
100%|██████████| 2/2 [00:00<00:00, 16.46it/s]
                                             

{'eval_loss': nan, 'eval_runtime': 8.891, 'eval_samples_per_second': 1.012, 'eval_steps_per_second': 0.225, 'epoch': 0}


...

11/16/2025 11:18:38 - INFO - __main__ - Saving finetuned Multilingual T3 model weights for ChatterboxMultilingualTTS...
11/16/2025 11:18:59 - INFO - __main__ - Finetuned Multilingual T3 model weights saved to ../models/checkpoints/chatterbox_finetuned_swedish/t3_mtl23ls_v2.safetensors
11/16/2025 11:19:10 - INFO - __main__ - Full model components structured in ../models/checkpoints/chatterbox_finetuned_swedish
11/16/2025 11:19:10 - INFO - __main__ - *** Evaluating Multilingual T3 model ***


***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =    10.6647
  train_runtime            = 0:17:12.82
  train_samples_per_second =      0.765
  train_steps_per_second   =      0.097


/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
100%|██████████| 2/2 [00:00<00:00, 36.55it/s]
11/16/2025 11:19:13 - INFO - __main__ - Multilingual finetuning script finished.


***** eval metrics *****
  epoch                   =       10.0
  eval_loss               =        nan
  eval_runtime            = 0:00:02.69
  eval_samples_per_second =      3.344
  eval_steps_per_second   =      0.743


CompletedProcess(args=['python3', '../scripts/finetune_mtl.py', '--output_dir', '../models/checkpoints/chatterbox_finetuned_swedish', '--local_model_dir', '/nobackup/joeri765/projects/personalized-natural-interface/TTS/chatterbox/models/base/chatterbox', '--metadata_file', '../data/david/metadata.txt', '--dataset_dir', '../data/david', '--train_split_name', 'train', '--eval_split_size', '0.1', '--num_train_epochs', '10', '--per_device_train_batch_size', '2', '--gradient_accumulation_steps', '4', '--learning_rate', '1e-5', '--warmup_steps', '200', '--weight_decay', '0.01', '--max_grad_norm', '1.0', '--logging_steps', '10', '--evaluation_strategy', 'epoch', '--save_strategy', 'epoch', '--save_total_limit', '5', '--report_to', 'tensorboard', '--do_train', '--do_eval', '--dataloader_pin_memory', 'False', '--eval_on_start', 'True', '--label_names', 'labels_speech', '--text_column_name', 'text_scribe', '--language_id', 'sv', '--dataloader_num_workers', '0', '--seed', '42'], returncode=0)

## Create Chatterbox-MTL model from finetuned checkpoint

In [13]:
# Combine base + latest HF checkpoint into a runnable folder, then load.
@contextlib.contextmanager
def force_cpu_load():
    from torch.serialization import load as _orig_load
    def _cpu_load(*args, **kwargs):
        kwargs["map_location"] = torch.device("cpu")
        return _orig_load(*args, **kwargs)
    old = torch.load
    torch.load = _cpu_load
    try:
        yield
    finally:
        torch.load = old

In [16]:
# ---- Paths ----
ft_root = Path("../models/checkpoints/chatterbox_finetuned_swedish")

# ---- Latest HF checkpoint ----
ckpt_dirs = sorted(
    [p for p in ft_root.iterdir() if p.is_dir() and p.name.startswith("checkpoint-")],
    key=lambda p: int(p.name.split("-")[1])
)

ft_ckpt = ckpt_dirs[-1] if ckpt_dirs else ft_root
src_model = ft_ckpt / "model.safetensors"
assert src_model.exists(), f"Missing checkpoint file: {src_model}"
print("Using FT checkpoint:", src_model)

# ---- Build merged_for_infer from BASE assets ----
merged_dir = ft_root / "merged_for_infer"
if merged_dir.exists():
    shutil.rmtree(merged_dir)
merged_dir.mkdir(parents=True, exist_ok=True)

# ---- Copy base assets ----
copies = [
    ("ve.pt", "ve.pt"),
    ("s3gen.pt", "s3gen.pt"),
    ("conds.pt", "conds.pt"),
    ("Cangjie5_TC.json", "Cangjie5_TC.json"),
    ("mtl_tokenizer.json", "mtl_tokenizer.json"),
    ("tokenizer.json", "mtl_tokenizer.json"),
    ("grapheme_mtl_merged_expanded_v1.json", "mtl_tokenizer.json"),
    ("t3_mtl23ls_v2.safetensors", "t3_mtl23ls_v2.safetensors"),
    ("t3_23lang.safetensors", "t3_23lang.safetensors"),
]
for src_name, dst_name in copies:
    src = base_folder / src_name
    if src.exists():
        shutil.copy2(src, merged_dir / dst_name)

# ---- Require tokenizer and at least one T3 base file to be present ----
assert (merged_dir / "mtl_tokenizer.json").exists(), \
    f"Tokenizer not found in base_dir: {base_folder}"
t3_primary = None
for cand in ("t3_mtl23ls_v2.safetensors", "t3_23lang.safetensors"):
    if (merged_dir / cand).exists():
        t3_primary = cand
        break
assert t3_primary is not None, \
    f"No base T3 file found in {base_folder} (looked for t3_mtl23ls_v2.safetensors / t3_23lang.safetensors)."

# ---- Convert FT checkpoint → T3 weights and overwrite base T3
sd = load_file(str(src_model))  # dict[str, Tensor]
t3_sd = {k.split("t3.", 1)[1]: v for k, v in sd.items() if k.startswith("t3.")} \
        if any(k.startswith("t3.") for k in sd) else dict(sd)
assert any(k.startswith(("tfmr.layers.", "tfmr.embed_tokens")) for k in t3_sd), \
       "Converted state dict doesn't look like T3 weights."

# ---- Write to the base T3 filename and also to the other alias for compatibility ----
save_file(t3_sd, str(merged_dir / t3_primary))
alias = "t3_23lang.safetensors" if t3_primary == "t3_mtl23ls_v2.safetensors" else "t3_mtl23ls_v2.safetensors"
save_file(t3_sd, str(merged_dir / alias))

print("[OK] Wrote finetuned T3 to:", (merged_dir / t3_primary), "and", (merged_dir / alias))

Using FT checkpoint: ../models/checkpoints/chatterbox_finetuned_swedish/checkpoint-100/model.safetensors
[OK] Wrote finetuned T3 to: ../models/checkpoints/chatterbox_finetuned_swedish/merged_for_infer/t3_mtl23ls_v2.safetensors and ../models/checkpoints/chatterbox_finetuned_swedish/merged_for_infer/t3_23lang.safetensors


## Load models

In [17]:
def load_chatterbox_model(model_folder: str, device: str | None = None):
    """
    Load a ChatterboxMultilingualTTS model from a folder.
    Uses force_cpu_load to be safe across CUDA/MPS/CPU.
    """
    model_folder = Path(model_folder)

    # If device not given, fall back to global `device` if you have one
    if device is None:
        device = globals().get("device", "cpu")

    print(f"Loading model from {model_folder} on device={device} (deserializing on CPU)…")

    with force_cpu_load():
        tts = ChatterboxMultilingualTTS.from_local(model_folder, device)

    if device in ("cuda", "mps"):
        tts.t3.to(device)
        tts.s3gen.to(device)
        tts.ve.to(device)
        tts.device = device

    print("Model loaded.")
    return tts

### Load base model

In [18]:
base_model = load_chatterbox_model(
    "../models/base/chatterbox"
)

Loading model from ../models/base/chatterbox on device=cpu (deserializing on CPU)…


/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


loaded PerthNet (Implicit) at step 250,000
Model loaded.


### Load finetuned model

In [19]:
# IMPORTANT: Make sure to create the finetuned model before running this cell
tts = load_chatterbox_model(
    "../models/checkpoints/chatterbox_finetuned_swedish/merged_for_infer"
)

Loading model from ../models/checkpoints/chatterbox_finetuned_swedish/merged_for_infer on device=cpu (deserializing on CPU)…
loaded PerthNet (Implicit) at step 250,000
Model loaded.


## Create, apply and save conditionals for a model

### Create conditionals

In [20]:
def create_voice_profile_from_wav(
    tts,
    wav_path: str,
    output_dir: str = "../models/voices",
    name: str | None = None,
    exaggeration: float = 0.5,
):
    """
    Extract voice conditionals from a reference wav and save to disk.
    Only switches to CPU when the model is on MPS (MPS cannot run prepare_conditionals).
    """
    wav_path = Path(wav_path)
    if not wav_path.exists():
        raise FileNotFoundError(f"Reference audio not found: {wav_path}")

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    if not name or not name.strip():
        name = wav_path.stem

    # Determine original device
    original_device = getattr(tts, "device", "cpu")

    # --- Only move to CPU if using MPS ---
    if original_device == "mps":
        print("[voice] MPS detected — switching to CPU for conditionals extraction.")
        tts.t3.to("cpu")
        tts.s3gen.to("cpu")
        tts.ve.to("cpu")
        tts.device = "cpu"

    # --- Extract conditionals ---
    tts.prepare_conditionals(str(wav_path), exaggeration=exaggeration)
    conds = tts.conds

    if conds is None:
        # restore model before raising
        if original_device == "mps":
            tts.t3.to(original_device)
            tts.s3gen.to(original_device)
            tts.ve.to(original_device)
            tts.device = original_device
        raise RuntimeError("prepare_conditionals() returned None — extraction failed.")

    # --- Save profile ---
    out_path = output_dir / f"{name}.pt"
    conds.save(out_path)
    print(f"[voice] Saved voice profile to: {out_path}")

    # --- Restore device if we switched ---
    if original_device == "mps":
        tts.t3.to("mps")
        tts.s3gen.to("mps")
        tts.ve.to("mps")
        tts.device = "mps"

    return out_path

In [21]:
voice_path = create_voice_profile_from_wav(
    tts=tts,
    wav_path="../data/david/soundfiles/david_sentence25.wav",
    output_dir="../models/voices",
    name="david",
)

[voice] Saved voice profile to: ../models/voices/david.pt


### Apply conditionals to a model

In [22]:
def apply_voice_profile_from_path(tts, conds_path: str):
    """
    Load Conditionals (.pt) from disk and apply them to a ChatterboxMultilingualTTS model.
    Automatically uses the device the model is already on.
    """
    conds_path = Path(conds_path)
    if not conds_path.exists():
        raise FileNotFoundError(f"Voice profile not found: {conds_path}")

    # Determine the model's device (fallback to CPU if somehow undefined)
    device = getattr(tts, "device", "cpu")

    # Load & move to that device
    conds = Conditionals.load(str(conds_path), map_location=device).to(device)

    # Attach to model
    tts.conds = conds

    print(f"[voice] Applied voice profile from {conds_path} → device={device}")
    return tts

In [23]:
apply_voice_profile_from_path(
    tts,
    "../models/voices/david.pt",
)

[voice] Applied voice profile from ../models/voices/david.pt → device=cpu


## Genereate soundfiles

In [29]:
def split_sentences(text):
    parts = re.split(r'([\.!?])', text)
    sents = ["".join(parts[i:i+2]).strip() for i in range(0, len(parts), 2)]
    return [s for s in sents if s]

def generate_chunked(tts, text, **gen_kwargs):
    wavs = []
    for s in split_sentences(text):
        w = tts.generate(s, **gen_kwargs)
        w = torch.as_tensor(w)
        if w.ndim == 1: w = w.unsqueeze(0)
        elif w.shape[0] > w.shape[-1]: w = w.t()
        wavs.append(w.to(torch.float32).contiguous().clamp_(-1,1).cpu())
    return torch.cat(wavs, dim=-1)    

def synthesize_with_model(
    tts,
    text: str,
    output_dir: str = "../output",
    language: str = "sv",
    exaggeration: float = 0.5,
    cfg_weight: float = 0.5,
    filename: str = "", # if empty, name will be timebased
):
    """
    Use an already loaded tts model to synthesize text.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # ---- Generate waveform ----
    wav = generate_chunked(
        tts,
        text,
        language_id=language,
        exaggeration=exaggeration,
        cfg_weight=cfg_weight,
    )

    # ---- Determine filename ----
    if not filename: 
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"{timestamp}_{language.lower()}.wav"
    else:
        # ensure extension
        if not filename.lower().endswith(".wav"):
            filename += ".wav"

    out_path = output_dir / filename
    
    # ---- set sample rate ----
    sr = getattr(tts, "sr", getattr(tts, "sample_rate", 24000))

    # ---- Save ----
    ta.save(str(out_path), wav.cpu(), sr)

    print("Saved:", out_path)
    return wav, sr, out_path

### Examples

In [25]:
# ---- Text to synthesize ----
text_sv = "Hej, det här är ett test för att utvärdera vår nya anpassade TTS-modell."

synthesize_with_model(
    tts=base_model,
    text=text_sv,
    output_dir="../examples",
    language="sv",
    exaggeration=0.5,
    cfg_weight=0.5,
    filename="tts-base"
)

/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Sampling:  14%|█▍  

Saved: ../examples/tts-base.wav


(tensor([[-1.4995e-09, -1.4760e-09, -4.9755e-10,  ..., -2.9102e-06,
          -1.3889e-05, -1.6783e-06]]),
 24000,
 PosixPath('../examples/tts-base.wav'))

In [ ]:
## Evaluation

In [32]:
import re
import numpy as np
import whisper
from jiwer import cer as jiwer_cer
from jiwer import wer as jiwer_wer

# Normalize_text: make it lowercase, remove punctuation and collapse multiple spaces
def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^\wåäöéüõçñ ]+", "", s, flags=re.UNICODE)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# CER
def cer(ref: str, hyp: str) -> float:
    ref_norm = normalize_text(ref)
    hyp_norm = normalize_text(hyp)
    return jiwer_cer(ref_norm, hyp_norm)

# WER
def wer(ref: str, hyp: str) -> float:
    ref_norm = normalize_text(ref)
    hyp_norm = normalize_text(hyp)
    return jiwer_wer(ref_norm, hyp_norm)

# Whisper loading
print("Loading Whisper model (medium)...")
whisper_model = whisper.load_model("medium")
print("Whisper model loaded.")

def transcribe_audio(audio_path: str, language: str = "sv") -> str:
    """Transcribe TTS audio using Whisper (Swedish)."""
    result = whisper_model.transcribe(audio_path, language=language, task="transcribe")
    return result["text"]


Loading Whisper model (medium)...
Whisper model loaded.


In [34]:
from pathlib import Path

eval_sentences = [
    "Det här är ett test av chatterbox TTS finetuning.",
    "Idag ska vi utvärdera svensk talsyntes.",
    "Rösten ska vara tydlig även när meningarna blir längre.",
    "Nu prövar vi hur modellen hanterar olika vokalljud.",
    "Sju sjuka sjömän sköttes av sju sköna sjuksköterskor.",
    "Kan modellen uttala både korta och långa vokaler korrekt?",
    "Här kommer en mening med siffror: tre, tolv och tjugotre.",
    "Hur låter prosodin när tempot ökar lite grann?",
    "Jag vill höra om uttalet förändras när frågetecken används?",
    "Till sist testar vi en riktigt lång mening för att se om modellen klarar att behålla ett naturligt flyt och en stabil volym genom hela uttalet."
]

print("Using", len(eval_sentences), "evaluation sentences.")

# Output directories
ft_dir   = Path("../examples/eval_audio/finetuned")
base_dir = Path("../examples/eval_audio/base")

ft_dir.mkdir(parents=True, exist_ok=True)
base_dir.mkdir(parents=True, exist_ok=True)


# Synthesize audio 
for i, text in enumerate(eval_sentences):

    # Finetuned
    finetune_filename = f"finetune_{i:03d}"
    synthesize_with_model(
        tts=tts,
        text=text,
        output_dir=str(ft_dir),
        language="sv",
        exaggeration=0.5,
        cfg_weight=0.5,
        filename=finetune_filename,
    )

    # Baseline
    base_filename = f"base_{i:03d}"
    synthesize_with_model(
        tts=base_model,
        text=text,
        output_dir=str(base_dir),
        language="sv",
        exaggeration=0.5,
        cfg_weight=0.5,
        filename=base_filename,
    )

# CER evaluation 
print("\n=== CER and WER EVALUATION ===")

base_scores_cer = []
ft_scores_cer   = []

base_scores_wer = []
ft_scores_wer   = []

for i, ref_text in enumerate(eval_sentences):
    finetune_filename = f"finetune_{i:03d}.wav"
    base_filename = f"base_{i:03d}.wav"
    base_path = base_dir / base_filename
    ft_path   = ft_dir   / finetune_filename

    base_transcribed = transcribe_audio(str(base_path))
    ft_transcribed   = transcribe_audio(str(ft_path))

    base_cer = cer(ref_text, base_transcribed)
    ft_cer   = cer(ref_text, ft_transcribed)
    
    base_wer = wer(ref_text, base_transcribed)
    ft_wer   = wer(ref_text, ft_transcribed)

    base_scores_cer.append(base_cer)
    ft_scores_cer.append(ft_cer)
    
    base_scores_wer.append(base_wer)
    ft_scores_wer.append(ft_wer)

    print(f"\n[{i:03d}] -----------------------------")
    print(f"REF:       {ref_text}")
    print(f"BASE:  {base_transcribed}")
    print(f"FT:    {ft_transcribed}")
    print(f"CER base:  {base_cer:.3f}")
    print(f"CER ft:    {ft_cer:.3f}")
    print(f"WER base:  {base_wer:.3f}")
    print(f"WER ft:    {ft_wer:.3f}")


# Summary
base_cer_avg = float(np.mean(base_scores_cer))
ft_cer_vg   = float(np.mean(ft_scores_cer))

base_wer_avg = float(np.mean(base_scores_wer))
ft_wer_avg   = float(np.mean(ft_scores_wer))

print("\n===== CER and WER SUMMARY =====")
print(f"Base CER: {base_cer_avg:.3f}")
print(f"Finetuned CER: {ft_cer_vg:.3f}")
print(f"Base WER: {base_wer_avg:.3f}")
print(f"Finetuned WER: {ft_wer_avg:.3f}")





Using 10 evaluation sentences.


/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/nobackup/joeri765/venvs/pni-py310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(
Sampling:  10%|█         | 101/1000 [00:06<00:55, 16.27it/s]


Saved: ../examples/eval_audio/finetuned/finetune_000.wav


Sampling:   9%|▉         | 92/1000 [00:05<00:57, 15.74it/s]


Saved: ../examples/eval_audio/base/base_000.wav


...

Saved: ../examples/eval_audio/base/base_008.wav


Sampling:  20%|██        | 205/1000 [00:14<00:55, 14.27it/s]


Saved: ../examples/eval_audio/finetuned/finetune_009.wav


Sampling:  16%|█▌        | 160/1000 [00:11<00:58, 14.45it/s]


Saved: ../examples/eval_audio/base/base_009.wav

=== CER and WER EVALUATION ===

[000] -----------------------------
REF:       Det här är ett test av chatterbox TTS finetuning.
BASE:   Det här är Testar Jack the Box TTS för inertuning.
FT:     Det här är ett test av Chatterbox TTS fina tuning.
CER base:  0.333
CER ft:    0.042
WER base:  0.667
WER ft:    0.222

[001] -----------------------------
REF:       Idag ska vi utvärdera svensk talsyntes.
BASE:   Idag ska vi utvärdera svensk talsyntes.
FT:     Idag ska vi utvärdera svensk talsyntes.
CER base:  0.000
CER ft:    0.000
WER base:  0.000
WER ft:    0.000

[002] -----------------------------
REF:       Rösten ska vara tydlig även när meningarna blir längre.
BASE:   Rösten ska vara tydlig även när meningarna blir längre.
FT:     Hösten ska vara tydlig även när meningen blir längre.
CER base:  0.000
CER ft:    0.074
WER base:  0.000
WER ft:    0.222

[003] -----------------------------
REF:       Nu prövar vi hur modellen hanterar oli